In [6]:
from pyspark.sql import Row
def returnRow(line):
    fields=line.split(",")
    return Row(ID=int(fields[0]),name=str(fields[1].encode("UTF-8")),age=int(fields[2]),numFriends=int(fields[3]))
rdd=sc.textFile("file:///E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/fakefriends.csv")
df=rdd.map(returnRow)
df=sqlCtx.createDataFrame(df)
df.cache()
df.createOrReplaceTempView("people")

In [7]:
teenagers=sqlContext.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")
teenagers.show(2)

+---+---+----------+----------+
| ID|age|      name|numFriends|
+---+---+----------+----------+
| 21| 19|  b'Miles'|       268|
| 52| 19|b'Beverly'|       269|
+---+---+----------+----------+
only showing top 2 rows



In [8]:
df.groupBy("age").count().orderBy("age").show()

+---+-----+
|age|count|
+---+-----+
| 18|    8|
| 19|   11|
| 20|    5|
| 21|    8|
| 22|    7|
| 23|   10|
| 24|    5|
| 25|   11|
| 26|   17|
| 27|    8|
| 28|   10|
| 29|   12|
| 30|   11|
| 31|    8|
| 32|   11|
| 33|   12|
| 34|    6|
| 35|    8|
| 36|   10|
| 37|    9|
+---+-----+
only showing top 20 rows



In [20]:
def loadMovieNames():
    movieNamesRows = []
    with open("E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/ml-100k/ml-100k/u.item") as f:
        for line in f:
            fields = line.split('|')
            movieNamesRows.append(Row(movieId=int(fields[0]),movieName=fields[1]))
    return movieNamesRows

def returnMovieRow(line):
    fields=line.split()
    return Row(userId=int(fields[0]),movieId=int(fields[1]),timestamp=int(fields[2]))

movieNameRdd = sc.parallelize(loadMovieNames())
rdd = sc.textFile("file:///E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/ml-100k/ml-100k/u.data")
rows=rdd.map(returnMovieRow)
movieDF=sqlContext.createDataFrame(rows)
movieDF.createOrReplaceTempView("movieRatings")
movieDF.cache()
movieNameDF=sqlContext.createDataFrame(movieNameRdd)
movieNameDF.createOrReplaceTempView("movieNames")
movieNameDF.cache()                

DataFrame[movieId: bigint, movieName: string]

In [30]:
joined=movieDF.join(movieNameDF,movieDF.movieId==movieNameDF.movieId)
joined.groupBy("movieName").count().orderBy("count",ascending=False).show()

+--------------------+-----+
|           movieName|count|
+--------------------+-----+
|    Star Wars (1977)|  583|
|      Contact (1997)|  509|
|        Fargo (1996)|  508|
|Return of the Jed...|  507|
|    Liar Liar (1997)|  485|
|English Patient, ...|  481|
|       Scream (1996)|  478|
|    Toy Story (1995)|  452|
|Air Force One (1997)|  431|
|Independence Day ...|  429|
|Raiders of the Lo...|  420|
|Godfather, The (1...|  413|
| Pulp Fiction (1994)|  394|
|Twelve Monkeys (1...|  392|
|Silence of the La...|  390|
|Jerry Maguire (1996)|  384|
|  Chasing Amy (1997)|  379|
|    Rock, The (1996)|  378|
|Empire Strikes Ba...|  367|
|Star Trek: First ...|  365|
+--------------------+-----+
only showing top 20 rows

